<a href="https://colab.research.google.com/github/agupte87/LearningPyspark/blob/master/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd ..
%mkdir spark

/


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [3]:
!wget -P /spark/ https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

--2021-07-21 14:41:00--  https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 64.50.233.100, 140.211.166.134, 64.50.236.52, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|64.50.233.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224762914 (214M) [application/x-gzip]
Saving to: ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’

spark-3.0.3-bin-had 100%[===================>] 214.35M  50.0MB/s    in 7.9s    

2021-07-21 14:41:08 (27.0 MB/s) - ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’ saved [224762914/224762914]



In [4]:
!tar xf /spark/spark-3.0.3-bin-hadoop3.2.tgz 
%mv spark-3.0.3-bin-hadoop3.2 /spark/




In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/spark/spark-3.0.3-bin-hadoop3.2"

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
%cd /content/
%mkdir /chapterData 
%cd /chapterData
%mkdir retail-data
%cd retail-data/
%mkdir by-day
%cd by-day

/content
/chapterData
/chapterData/retail-data
/chapterData/retail-data/by-day


In [9]:
%pwd


'/chapterData/retail-data/by-day'

In [10]:

!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-01.csv'
!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-02.csv'
!wget -P  / 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-03.csv'

--2021-07-21 14:41:21--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-01.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275001 (269K) [text/plain]
Saving to: ‘2010-12-01.csv’

010-12-01.csv       100%[===================>] 268.56K  --.-KB/s    in 0.009s  

2021-07-21 14:41:21 (28.3 MB/s) - ‘2010-12-01.csv’ saved [275001/275001]

--2021-07-21 14:41:21--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-02.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected

In [11]:
%mv /chapterData /content/

# New Section

In [12]:
staticDF = spark.read.format("csv")\
.option("inferSchema", "true")\
.option("header", "true")\
.load("/content/chapterData/retail-data/by-day/*.csv")

In [17]:
staticSchema = staticDF.schema

In [13]:
staticDF.show(10)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [14]:
dayOfLargePurchase = staticDF.groupby("InvoiceDate").sum("UnitPrice").withColumnRenamed("sum(UnitPrice)", "SumOFUnitPrice")
dayOfLargePurchase.sort(dayOfLargePurchase.InvoiceDate.desc()).take(10)

[Row(InvoiceDate='2010-12-03 17:28:00', SumOFUnitPrice=92.47),
 Row(InvoiceDate='2010-12-03 17:21:00', SumOFUnitPrice=1.45),
 Row(InvoiceDate='2010-12-03 17:20:00', SumOFUnitPrice=153.11),
 Row(InvoiceDate='2010-12-03 16:50:00', SumOFUnitPrice=0.0),
 Row(InvoiceDate='2010-12-03 16:45:00', SumOFUnitPrice=7.5),
 Row(InvoiceDate='2010-12-03 16:44:00', SumOFUnitPrice=24.599999999999998),
 Row(InvoiceDate='2010-12-03 16:37:00', SumOFUnitPrice=0.0),
 Row(InvoiceDate='2010-12-03 16:36:00', SumOFUnitPrice=0.0),
 Row(InvoiceDate='2010-12-03 16:35:00', SumOFUnitPrice=37.41),
 Row(InvoiceDate='2010-12-03 16:21:00', SumOFUnitPrice=60.540000000000006)]

In [15]:
from pyspark.sql.functions import window , column , desc , col 
staticDataFrame = staticDF.selectExpr("CustomerID", "(UnitPrice*Quantity) as total_purchase_value", "InvoiceDate").groupBy(\
    col("CustomerId"),\
    window(col("InvoiceDate"),"1 day"))\
.sum("total_purchase_value")\
.withColumnRenamed("sum(total_purchase_value)", "total_purchase_value")\
.show(10)



+----------+--------------------+--------------------+
|CustomerId|              window|total_purchase_value|
+----------+--------------------+--------------------+
|   13408.0|[2010-12-01 00:00...|  1024.6800000000003|
|   17460.0|[2010-12-01 00:00...|                19.9|
|   15235.0|[2010-12-01 00:00...|                79.5|
|   13295.0|[2010-12-02 00:00...|               -3.25|
|   12841.0|[2010-12-03 00:00...|              294.25|
|   12779.0|[2010-12-03 00:00...|              248.16|
|   14156.0|[2010-12-02 00:00...|                -7.5|
|   13520.0|[2010-12-03 00:00...|               303.4|
|   18239.0|[2010-12-02 00:00...|   438.0999999999999|
|   17905.0|[2010-12-01 00:00...|  201.74999999999997|
+----------+--------------------+--------------------+
only showing top 10 rows



In [18]:
readData = spark.readStream.schema(staticSchema)\
.option("maxFilesTrigger",1)\
.option("header", "true")\
.format("csv")\
.load("/content/chapterData/retail-data/by-day/*.csv")

In [19]:
readData.isStreaming

True

In [30]:
purchaseCustomerPerHour = readData.selectExpr("CustomerId", "(UnitPrice*Quantity) as Total_Cost", "InvoiceDate")\
.groupBy(col("CustomerId")\
,window(col("InvoiceDate"), "1 day"))\
.sum("Total_Cost")

In [31]:
purchaseCustomerPerHr= purchaseCustomerPerHour.writeStream\
.format("memory")\
.queryName("Customer_Purchase")\
.outputMode("complete")\
.start()

IllegalArgumentException: ignored

In [33]:
spark.sql("select * from Customer_Purchase").show(10)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(Total_Cost)|
+----------+--------------------+------------------+
|   17460.0|[2010-12-01 00:00...|              19.9|
|   13408.0|[2010-12-01 00:00...|1024.6800000000003|
|   15235.0|[2010-12-01 00:00...|              79.5|
|   12841.0|[2010-12-03 00:00...|            294.25|
|   13295.0|[2010-12-02 00:00...|             -3.25|
|   12779.0|[2010-12-03 00:00...|            248.16|
|   14156.0|[2010-12-02 00:00...|              -7.5|
|   13520.0|[2010-12-03 00:00...|             303.4|
|   18239.0|[2010-12-02 00:00...| 438.0999999999999|
|   12712.0|[2010-12-03 00:00...|175.53000000000003|
+----------+--------------------+------------------+
only showing top 10 rows



In [36]:
staticDataFrame.printSchema()

AttributeError: ignored